In [20]:
import sys
sys.path[:0] = ['../../ref/sw/compiler', '../basics']

from parser import IRParser
from backend import CompilerBackend
from asm_interp import AsmInterp


In [21]:
p = IRParser()
cb = CompilerBackend()
ai = AsmInterp()

In [26]:
cb.funcs(p('''
    /* main func */
    main(0) = if pos(pow2($0)) then 1 else 0

    /* power ^2 function */
    pow2(1) = $0 * $0

    /* positive number */
    pos(1) = if 0 < $0 then 1 else 0
'''))

In [27]:
cb.code

['main',
 ('PUSH', 'main:0'),
 ('PUSH', 'main:1'),
 ('PUSH', 2),
 ('POP', 2),
 ('ALU', 'SUB'),
 ('JMP', 'pow2'),
 'main:1',
 ('JMP', 'pos'),
 'main:0',
 ('POP', 1),
 ('JZ', 'main:2'),
 ('PUSH', 1),
 ('JMP', 'main:3'),
 'main:2',
 ('PUSH', 0),
 'main:3',
 ('POP', 2),
 ('RET', 1),
 'pow2',
 ('DUP', 0),
 ('DUP', 1),
 ('POP', 2),
 ('ALU', 'MUL'),
 ('YANK', (1, 1)),
 ('POP', 2),
 ('RET', 1),
 'pos',
 ('PUSH', 0),
 ('DUP', 1),
 ('POP', 2),
 ('ALU', 'LT'),
 ('POP', 1),
 ('JZ', 'pos:0'),
 ('PUSH', 1),
 ('JMP', 'pos:1'),
 'pos:0',
 ('PUSH', 0),
 'pos:1',
 ('YANK', (1, 1)),
 ('POP', 2),
 ('RET', 1),
 'main',
 ('PUSH', 'main:4'),
 ('PUSH', 'main:5'),
 ('PUSH', 2),
 ('POP', 2),
 ('ALU', 'SUB'),
 ('JMP', 'pow2'),
 'main:5',
 ('JMP', 'pos'),
 'main:4',
 ('POP', 1),
 ('JZ', 'main:6'),
 ('PUSH', 1),
 ('JMP', 'main:7'),
 'main:6',
 ('PUSH', 0),
 'main:7',
 ('POP', 2),
 ('RET', 1),
 'pow2',
 ('DUP', 0),
 ('DUP', 1),
 ('POP', 2),
 ('ALU', 'MUL'),
 ('YANK', (1, 1)),
 ('POP', 2),
 ('RET', 1),
 'pos',
 ('PU